In [8]:
# ==================================================
# Azure Blob Storage Demo
# ==================================================
import sys, os
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("."))

from app.clients.blob_storage import BlobStorageClient
# Initialisierung
client = BlobStorageClient()

# Testdatei Erzeugung
test_file_path = "test_upload.txt"
with open(test_file_path, "w") as f:
    f.write("Testinhalt - Azure Blob Storage funktioniert!")

# Datei hochladen
upload_ok = client.upload_file(test_file_path, "upload_test_notebook.txt")
print("Upload erfolgreich: ", upload_ok)

# Dateien herunterladen
download_path = "download_test_notebook.txt"
download_ok = client.download_file("upload_test_notebook.txt", download_path)
print ("/nDownload erfolgreich: ", download_ok)

# Lokalen Inhalt Überprüfung
with open(download_path, "r") as f:
    content = f.read()
print("/nInhalt der heruntergeladenen Dateien: ")
print(content)

Upload erfolgreich:  True
/nDownload erfolgreich:  True
/nInhalt der heruntergeladenen Dateien: 
Testinhalt - Azure Blob Storage funktioniert!


In [43]:
# =========================================================
# Löschung vom bestehenden Index
# =========================================================
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchableField

from app.config import AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_KEY, AZURE_SEARCH_INDEX


index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    credential=AzureKeyCredential(AZURE_SEARCH_KEY)
)

# Wenn alter Index vorhanden ist, es wird zuerst gelöscht
try:
    index_client.delete_index(AZURE_SEARCH_INDEX)
    print(f"Alter Index ({AZURE_SEARCH_INDEX}) wurde gelöscht.")
except Exception:
    pass



Alter Index (documents) wurde gelöscht.


In [44]:
# ===========================================================
# Erstellung eines neuen Indexes
# ============================================================
fields = [
    SimpleField(name="id", type="Edm.String", key=True),
    SearchableField(name="title", type="Edm.String", retrievable=True),
    SearchableField(name="content", type="Edm.String", retrievable=True)
]

index = SearchIndex(name=AZURE_SEARCH_INDEX, fields=fields)

try:
    index_client.create_index(index)
    print(f"Index '{AZURE_SEARCH_INDEX}' erfolgreich initialisiert.")
except Exception as e:
    print(f"Fehler ist bei der Erstellung des Indexes aufgetreten: {e}")

Index 'documents' erfolgreich initialisiert.


In [47]:
# ========================================================
# Das Hochladen von Beispieldokumenten
# ========================================================
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential


search_client = SearchClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX,
    credential=AzureKeyCredential(AZURE_SEARCH_KEY)
)

docs = [
     {"id": "1", "title": "Azure Cognitive Search Test", "content": "Dies ist ein Testdokument."},
    {"id": "2", "title": "Künstliche Intelligenz", "content": "Azure Search ermöglicht KI-basierte Suche."},
    {"id": "3", "title": "Cloud Computing", "content": "Speicherung und Verarbeitung in der Cloud."},
    {"id": "4", "title": "Python Programmierung", "content": "Python ist eine beliebte Programmiersprache."},
    {"id": "5", "title": "FastAPI", "content": "Erstellung von APIs mit FastAPI."}
]

search_client.upload_documents(documents=docs)
print("Dokumente erfolgreich hochgeladen.")

Dokumente erfolgreich hochgeladen.


In [49]:
# ========================================================
# Cognitive Search Demo
# =========================================================
from app.clients.cognitive_search import CoginitiveSearchClient
from app.config import AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_KEY, AZURE_SEARCH_INDEX

# Initialisierung des Clients
try:
    search_client = CoginitiveSearchClient()
    print("Cognitive Search Client wurde erfolgreich initialisert.")
except Exception as e:
    print("Fehler bei der Initialisierung: {e}")

query = "Azure"

try:
    results = search_client.search_documents(query=query, top=5)
    print(f"Gefundene Dokumente: {len(results)}")
    for doc in results:
        print(doc)
except Exception as e:
    print(f"Fehler bei der Suche: {e}")

Cognitive Search Client wurde erfolgreich initialisert.
Gefundene Dokumente: 2
{'content': 'Azure Search ermöglicht KI-basierte Suche.', 'title': 'Künstliche Intelligenz', 'id': '2', '@search.score': 1.2051069, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'content': 'Dies ist ein Testdokument.', 'title': 'Azure Cognitive Search Test', 'id': '1', '@search.score': 1.0386478, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}


In [56]:
# ===========================================================================
# Azure OpenAI Demo
# ===========================================================================
from app.clients.openai_client import OpenAIClientWrapper

client = OpenAIClientWrapper()
prompt = "Sag hallo"
response = client.generate_answer(prompt=prompt, max_tokens=50)
print(response)

ResourceNotFoundError: (404) Resource not found
Code: 404
Message: Resource not found